# basic code for getting a single NBA game PlayByPlay, sub stints

This is a repository for some raw code to get various elements of a single game PlayByPlay data, mostly keeping an eye towards game stints (periods of play with the same players on the court, aka between substitutions). This makes heavy use of the nba_api package found here: https://github.com/swar/nba_api

In [117]:
#!pip install nba_api

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [118]:
"""retrieve team and game id, team roster"""

import pandas as pd

# get celtics_id from teams list
from nba_api.stats.static import teams
nba_teams = teams.get_teams()
celtics = [team for team in nba_teams if team['abbreviation'] == 'BOS'][0]
celtics_id = celtics['id']
del(nba_teams, celtics)

# get dataframe with all of celtics games this season (mostly for game ids and HvV, add
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id,
                                               season_nullable=Season.default,
                                               season_type_nullable=SeasonType.regular)
games = gamefinder.get_data_frames()[0]
del(gamefinder)

# sort the games and add home or visitor, remove extra columns
sorted_games = games[['GAME_ID', 'GAME_DATE', 'MATCHUP']].sort_values(by=['GAME_DATE'])
sorted_games['GAME_NUM'] = range(1, len(sorted_games) + 1)
sorted_games["HvV"] = sorted_games["MATCHUP"].map(
    lambda x: "HOME" if "vs." in x else "VISITOR" if "@" in x else NameError)
del(games)

# get the celtics roster
from nba_api.stats.endpoints import playerindex
BOS_roster = playerindex.PlayerIndex(team_id_nullable=celtics_id).get_data_frames()[0]

# get the game_id for a single game of the year
single_game_id = sorted_games.loc[sorted_games['GAME_NUM'] == 79, 'GAME_ID'].item()

In [119]:
sorted_games.head()

,GAME_ID,GAME_DATE,MATCHUP,GAME_NUM,HvV
81,0022300065,2023-10-25,BOS @ NYK,1,VISITOR
80,0022300080,2023-10-27,BOS vs. MIA,2,HOME
79,0022300103,2023-10-30,BOS @ WAS,3,VISITOR
78,0022300118,2023-11-01,BOS vs. IND,4,HOME
77,0022300136,2023-11-04,BOS @ BKN,5,VISITOR


In [120]:
BOS_roster.head()

,PERSON_ID,PLAYER_LAST_NAME,PLAYER_FIRST_NAME,PLAYER_SLUG,TEAM_SLUG,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,JERSEY_NUMBER,POSITION,HEIGHT,WEIGHT,COLLEGE,COUNTRY,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,ROSTER_STATUS,FROM_YEAR,TO_YEAR,PTS,REB,AST,STATS_TIMEFRAME
0,1629052,Brissett,Oshae,oshae-brissett,celtics,1610612738,Boston,Celtics,BOS,12,F-G,6-7,210,Syracuse,Canada,NaN,NaN,NaN,1.0,2019,2023,3.7,2.9,0.8,Season
1,1627759,Brown,Jaylen,jaylen-brown,celtics,1610612738,Boston,Celtics,BOS,7,G-F,6-6,223,California,USA,2016.0,1.0,3.0,1.0,2016,2023,23.0,5.5,3.6,Season
2,1631120,Davison,JD,jd-davison,celtics,1610612738,Boston,Celtics,BOS,20,G,6-1,195,Alabama,USA,2022.0,2.0,53.0,1.0,2022,2023,2.0,1.3,1.3,Season
3,1630573,Hauser,Sam,sam-hauser,celtics,1610612738,Boston,Celtics,BOS,30,F,6-7,217,Virginia,USA,NaN,NaN,NaN,1.0,2021,2023,9.0,3.5,1.0,Season
4,201950,Holiday,Jrue,jrue-holiday,celtics,1610612738,Boston,Celtics,BOS,4,G,6-4,205,UCLA,USA,2009.0,1.0,17.0,1.0,2009,2023,12.5,5.4,4.8,Season


In [121]:
"""get the playbyplay for that game"""

from nba_api.stats.endpoints import playbyplayv2
pbp_df = playbyplayv2.PlayByPlayV2(single_game_id).get_data_frames()[0]

In [122]:
print(pbp_df.shape)
pbp_df.head()

(371, 34)


,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
0,0022301148,2,12,0,1,7:36 PM,12:00,None,Start of 1st Period (7:36 PM EST),None,None,None,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,0
1,0022301148,4,10,0,1,7:36 PM,12:00,Jump Ball Lopez vs. Tillman: Tip to G. Antetok...,None,None,None,None,4,201572,Brook Lopez,1.610613e+09,Milwaukee,Bucks,MIL,5,1630214,Xavier Tillman,1.610613e+09,Boston,Celtics,BOS,4,203507,Giannis Antetokounmpo,1.610613e+09,Milwaukee,Bucks,MIL,1
2,0022301148,7,1,52,1,7:37 PM,11:48,G. Antetokounmpo 2' Alley Oop Dunk (2 PTS) (Mi...,None,None,0 - 2,2,4,203507,Giannis Antetokounmpo,1.610613e+09,Milwaukee,Bucks,MIL,4,203114,Khris Middleton,1.610613e+09,Milwaukee,Bucks,MIL,0,0,None,NaN,None,None,None,1
3,0022301148,9,5,39,1,7:37 PM,11:35,None,None,White Step Out of Bounds Turnover (P1.T1),None,None,5,1628401,Derrick White,1.610613e+09,Boston,Celtics,BOS,0,0,None,NaN,None,None,None,1,0,None,NaN,None,None,None,1
4,0022301148,10,2,1,1,7:37 PM,11:29,MISS Lopez 26' 3PT Jump Shot,None,None,None,None,4,201572,Brook Lopez,1.610613e+09,Milwaukee,Bucks,MIL,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,1


In [123]:
"""modifying the playbyplay into a new df with new columns"""

# copy the df
pbp2_df = pbp_df.copy()

# add a column for time remaining (in seconds)
pbp2_df.insert(7, 'SECONDS_LEFT', (4-pbp2_df['PERIOD']) * 12 * 60 + pd.Series(map(int, pbp2_df['PCTIMESTRING'].str.partition(':')[0])) * 60 + pd.Series(map(int, pbp2_df['PCTIMESTRING'].str.partition(':')[2])))

# adding running tally of score (filling the first row, forward filling columns, replacing TIE with 0)
pbp2_df.loc[0, ['SCORE', 'SCOREMARGIN']] = '0 - 0', 0
pbp2_df[['SCORE', 'SCOREMARGIN']] = pbp2_df[['SCORE', 'SCOREMARGIN']].ffill()
pbp2_df.loc[pbp2_df['SCOREMARGIN'] == 'TIE', 'SCOREMARGIN'] = 0

In [124]:
print(pbp2_df.shape)
pbp2_df.head()

(371, 35)


,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,SECONDS_LEFT,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
0,0022301148,2,12,0,1,7:36 PM,12:00,2880,None,Start of 1st Period (7:36 PM EST),None,0 - 0,0,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,0
1,0022301148,4,10,0,1,7:36 PM,12:00,2880,Jump Ball Lopez vs. Tillman: Tip to G. Antetok...,None,None,0 - 0,0,4,201572,Brook Lopez,1.610613e+09,Milwaukee,Bucks,MIL,5,1630214,Xavier Tillman,1.610613e+09,Boston,Celtics,BOS,4,203507,Giannis Antetokounmpo,1.610613e+09,Milwaukee,Bucks,MIL,1
2,0022301148,7,1,52,1,7:37 PM,11:48,2868,G. Antetokounmpo 2' Alley Oop Dunk (2 PTS) (Mi...,None,None,0 - 2,2,4,203507,Giannis Antetokounmpo,1.610613e+09,Milwaukee,Bucks,MIL,4,203114,Khris Middleton,1.610613e+09,Milwaukee,Bucks,MIL,0,0,None,NaN,None,None,None,1
3,0022301148,9,5,39,1,7:37 PM,11:35,2855,None,None,White Step Out of Bounds Turnover (P1.T1),0 - 2,2,5,1628401,Derrick White,1.610613e+09,Boston,Celtics,BOS,0,0,None,NaN,None,None,None,1,0,None,NaN,None,None,None,1
4,0022301148,10,2,1,1,7:37 PM,11:29,2849,MISS Lopez 26' 3PT Jump Shot,None,None,0 - 2,2,4,201572,Brook Lopez,1.610613e+09,Milwaukee,Bucks,MIL,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,1


In [125]:
"""get the starters for each quarter, store in dictionary
https://github.com/rd11490/NBA_Tutorials/blob/master/players_on_court/players_on_court.py"""


# imports
import json
import pandas as pd
import urllib3
header_data  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

# endpoints, helper functions
def play_by_play_url(game_id):
    return "https://stats.nba.com/stats/playbyplayv2/?gameId={0}&startPeriod=0&endPeriod=14".format(game_id)
def advanced_boxscore_url(game_id, start, end):
    return "https://stats.nba.com/stats/boxscoretraditionalv2/?gameId={0}&startPeriod=0&endPeriod=14&startRange={1}&endRange={2}&rangeType=2".format(game_id, start, end)
http = urllib3.PoolManager()
def extract_data(url):
    print(url)
    r = http.request('GET', url, headers=header_data)
    resp = json.loads(r.data)
    results = resp['resultSets'][0]
    headers = results['headers']
    rows = results['rowSet']
    frame = pd.DataFrame(rows)
    frame.columns = headers
    return frame
def calculate_time_at_period(period):
    if period > 5:
        return (720 * 4 + (period - 5) * (5 * 60)) * 10
    else:
        return (720 * (period - 1)) * 10
def split_subs(df, tag):
    subs = df[[tag, 'PERIOD', 'EVENTNUM']]
    subs['SUB'] = tag
    subs.columns = ['PLAYER_ID', 'PERIOD', 'EVENTNUM', 'SUB']
    return subs

# actual function
def quartersstarters(game_id):
    game_id = game_id
    frame = extract_data(play_by_play_url(game_id))

    substitutionsOnly = frame[frame["EVENTMSGTYPE"] == 8][['PERIOD', 'EVENTNUM', 'PLAYER1_ID', 'PLAYER2_ID']]
    substitutionsOnly.columns = ['PERIOD', 'EVENTNUM', 'OUT', 'IN']

    subs_in = split_subs(substitutionsOnly, 'IN')
    subs_out = split_subs(substitutionsOnly, 'OUT')

    full_subs = pd.concat([subs_out, subs_in], axis=0).reset_index()[['PLAYER_ID', 'PERIOD', 'EVENTNUM', 'SUB']]
    first_event_of_period = full_subs.loc[full_subs.groupby(by=['PERIOD', 'PLAYER_ID'])['EVENTNUM'].idxmin()]
    players_subbed_in_at_each_period = first_event_of_period[first_event_of_period['SUB'] == 'IN'][
        ['PLAYER_ID', 'PERIOD', 'SUB']]

    periods = players_subbed_in_at_each_period['PERIOD'].drop_duplicates().values.tolist()

    frames = []
    for period in periods:
        low = calculate_time_at_period(period) + 5
        high = calculate_time_at_period(period + 1) - 5
        boxscore = advanced_boxscore_url(game_id, low, high)
        boxscore_players = extract_data(boxscore)[['PLAYER_NAME', 'PLAYER_ID', 'TEAM_ABBREVIATION']]
        boxscore_players['PERIOD'] = period

        players_subbed_in_at_period = players_subbed_in_at_each_period[
            players_subbed_in_at_each_period['PERIOD'] == period]

        joined_players = pd.merge(boxscore_players, players_subbed_in_at_period, on=['PLAYER_ID', 'PERIOD'], how='left')
        joined_players = joined_players[pd.isnull(joined_players['SUB'])][
            ['PLAYER_NAME', 'PLAYER_ID', 'TEAM_ABBREVIATION', 'PERIOD']]
        frames.append(joined_players)

    quarter_starters = pd.concat(frames)
    return quarter_starters

qs = quartersstarters('0022301148')

BOS_qs1 = list(qs.loc[(qs['TEAM_ABBREVIATION'] == 'BOS') & (qs['PERIOD'] == 1), 'PLAYER_ID'])
BOS_qs2 = list(qs.loc[(qs['TEAM_ABBREVIATION'] == 'BOS') & (qs['PERIOD'] == 2), 'PLAYER_ID'])
BOS_qs3 = list(qs.loc[(qs['TEAM_ABBREVIATION'] == 'BOS') & (qs['PERIOD'] == 3), 'PLAYER_ID'])
BOS_qs4 = list(qs.loc[(qs['TEAM_ABBREVIATION'] == 'BOS') & (qs['PERIOD'] == 4), 'PLAYER_ID'])
OPP_qs1 = list(qs.loc[(qs['TEAM_ABBREVIATION'] != 'BOS') & (qs['PERIOD'] == 1), 'PLAYER_ID'])
OPP_qs2 = list(qs.loc[(qs['TEAM_ABBREVIATION'] != 'BOS') & (qs['PERIOD'] == 2), 'PLAYER_ID'])
OPP_qs3 = list(qs.loc[(qs['TEAM_ABBREVIATION'] != 'BOS') & (qs['PERIOD'] == 3), 'PLAYER_ID'])
OPP_qs4 = list(qs.loc[(qs['TEAM_ABBREVIATION'] != 'BOS') & (qs['PERIOD'] == 4), 'PLAYER_ID'])

qs_dict = {'BOS_qs1':BOS_qs1, 'BOS_qs2':BOS_qs2, 'BOS_qs3':BOS_qs3, 'BOS_qs4':BOS_qs4, 'OPP_qs1':OPP_qs1, 'OPP_qs2':OPP_qs2, 'OPP_qs3':OPP_qs3, 'OPP_qs4':OPP_qs4}
del(BOS_qs1, BOS_qs2, BOS_qs3, BOS_qs4, OPP_qs1, OPP_qs2, OPP_qs3, OPP_qs4)
del(header_data, http, qs)

https://stats.nba.com/stats/playbyplayv2/?gameId=0022301148&startPeriod=0&endPeriod=14
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022301148&startPeriod=0&endPeriod=14&startRange=5&endRange=7195&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022301148&startPeriod=0&endPeriod=14&startRange=7205&endRange=14395&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022301148&startPeriod=0&endPeriod=14&startRange=14405&endRange=21595&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022301148&startPeriod=0&endPeriod=14&startRange=21605&endRange=28795&rangeType=2


In [126]:
print(qs_dict)

{'BOS_qs1': [1627759, 1628369, 1630214, 201950, 1628401], 'BOS_qs2': [1628369, 1630214, 201950, 1630202, 1630573], 'BOS_qs3': [1627759, 1628369, 1630214, 201950, 1628401], 'BOS_qs4': [1627759, 1628369, 1630202, 1630573, 1628436], 'OPP_qs1': [203114, 203507, 201572, 201976, 203081], 'OPP_qs2': [203507, 1626171, 1627736, 1626192, 1631260], 'OPP_qs3': [203114, 203507, 201572, 201976, 203081], 'OPP_qs4': [203114, 1626171, 1627736, 1626192, 1631260]}


In [127]:
"""from pbp2_df, find all the events that are subs or quarter delineations"""

# in pbp2_df, find all the subs/quarters, and make a new df with them (8 is sub, 12 is start of quarter)
subs_df = pbp2_df[(pbp2_df['EVENTMSGTYPE'] == 8) | (pbp2_df['EVENTMSGTYPE'] == 12)]
# add a column for sub_stints, where the group of subs all count as one (this will separate game stints)
subs_df.insert(3, 'subs_stint_bool', subs_df.EVENTNUM == subs_df.EVENTNUM.shift() + 1)
subs_df.insert(3, 'subs_stint', subs_df['subs_stint_bool'].eq(False).cumsum())

In [128]:
print(subs_df.shape)
subs_df.head()

(39, 37)


,GAME_ID,EVENTNUM,EVENTMSGTYPE,subs_stint,subs_stint_bool,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,SECONDS_LEFT,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
0,0022301148,2,12,1,False,0,1,7:36 PM,12:00,2880,None,Start of 1st Period (7:36 PM EST),None,0 - 0,0,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,0
51,0022301148,75,8,2,False,0,1,7:48 PM,4:21,2421,SUB: Portis FOR G. Antetokounmpo,None,None,11 - 26,15,4,203507,Giannis Antetokounmpo,1.610613e+09,Milwaukee,Bucks,MIL,4,1626171,Bobby Portis,1.610613e+09,Milwaukee,Bucks,MIL,0,0,None,NaN,None,None,None,0
52,0022301148,76,8,2,True,0,1,7:48 PM,4:21,2421,None,None,SUB: Pritchard FOR Tatum,11 - 26,15,5,1628369,Jayson Tatum,1.610613e+09,Boston,Celtics,BOS,5,1630202,Payton Pritchard,1.610613e+09,Boston,Celtics,BOS,0,0,None,NaN,None,None,None,0
53,0022301148,77,8,2,True,0,1,7:48 PM,4:21,2421,None,None,SUB: Hauser FOR Holiday,11 - 26,15,5,201950,Jrue Holiday,1.610613e+09,Boston,Celtics,BOS,5,1630573,Sam Hauser,1.610613e+09,Boston,Celtics,BOS,0,0,None,NaN,None,None,None,0
54,0022301148,78,8,2,True,0,1,7:48 PM,4:21,2421,None,None,SUB: Kornet FOR Tillman,11 - 26,15,5,1630214,Xavier Tillman,1.610613e+09,Boston,Celtics,BOS,5,1628436,Luke Kornet,1.610613e+09,Boston,Celtics,BOS,0,0,None,NaN,None,None,None,0


In [129]:
"""creating a df to hold each game_stint (everything that happens between substitutions)"""

# start a df that will hold all the game stints (lineups)
game_stints_df = pd.DataFrame(index=range(0, 100),
                              columns=['Stint_id', 'start_period', 'end_period', 'start_time', 'end_time', 'start_seconds', 'end_seconds', 'duration_s',
                                       'start_score', 'end_score', 'start_margin', 'end_margin', 'plusminus',
                                       'player1_id', 'player2_id', 'player3_id', 'player4_id', 'player5_id',
                                       'lineup_long', 'lineup_id_ingame',
                                       'player6_id', 'player7_id', 'player8_id', 'player9_id', 'player10_id',
                                       'OPP_lineup_long', 'OPP_lineup_id_ingame'])

#
for sub_stint in subs_df['subs_stint'].unique():
    sub_index = subs_df[subs_df['subs_stint'] == sub_stint].index[0]

    game_stints_df.loc[sub_stint - 1, ['Stint_id']] = sub_stint
    game_stints_df.loc[sub_stint - 1, ['start_period']] = subs_df.loc[sub_index, ['PERIOD']].item()
    game_stints_df.loc[sub_stint - 1, ['start_time']] = subs_df.loc[sub_index, ['PCTIMESTRING']].item()
    game_stints_df.loc[sub_stint - 1, ['start_seconds']] = subs_df.loc[sub_index, ['SECONDS_LEFT']].item()
    game_stints_df.loc[sub_stint - 1, ['start_score']] = subs_df.loc[sub_index, ['SCORE']].item()
    game_stints_df.loc[sub_stint - 1, ['start_margin']] = subs_df.loc[sub_index, ['SCOREMARGIN']].item()

    # subs portion (manually putting in the quarter starters whenever there's a new quarter)
    if subs_df.loc[sub_index, ['EVENTMSGTYPE']].item() == 12:
        period = subs_df.loc[sub_index, ['PERIOD']].item()

        game_stints_df.loc[sub_stint - 1, ['lineup_long']] = "-".join(map(str, qs_dict[f"BOS_qs{period}"]))
        game_stints_df.loc[sub_stint - 1, ['player1_id', 'player2_id', 'player3_id', 'player4_id', 'player5_id']] = qs_dict[f"BOS_qs{period}"]

        game_stints_df.loc[sub_stint - 1, ['OPP_lineup_long']] = "-".join(map(str, qs_dict[f"OPP_qs{period}"]))
        game_stints_df.loc[sub_stint - 1, ['player6_id', 'player7_id', 'player8_id', 'player9_id', 'player10_id']] = qs_dict[f"OPP_qs{period}"]

    elif subs_df.loc[sub_index, ['EVENTMSGTYPE']].item() == 8:
        new_lineup_long_str = game_stints_df.loc[sub_stint - 2, ['lineup_long']].item()
        new_OPP_lineup_long_str = game_stints_df.loc[sub_stint - 2, ['OPP_lineup_long']].item()

        for row in subs_df[subs_df['subs_stint'] == sub_stint].index:
            sub_out = subs_df.loc[row, ['PLAYER1_ID']].item()
            sub_in = subs_df.loc[row, ['PLAYER2_ID']].item()
            if sub_out.astype(str) in new_lineup_long_str:
                new_lineup_long_str = new_lineup_long_str.replace(sub_out.astype(str), sub_in.astype(str))
            elif sub_out.astype(str) in new_OPP_lineup_long_str:
                new_OPP_lineup_long_str = new_OPP_lineup_long_str.replace(sub_out.astype(str), sub_in.astype(str))

        new_lineup_long_str.split("-")
        lineup_IDs = [int(x) for x in new_lineup_long_str.split("-")]
        game_stints_df.loc[sub_stint - 1, ['player1_id', 'player2_id', 'player3_id', 'player4_id', 'player5_id']] = lineup_IDs
        game_stints_df.loc[sub_stint - 1, ['lineup_long']] = new_lineup_long_str

        new_OPP_lineup_long_str.split("-")
        OPP_lineup_IDs = [int(x) for x in new_OPP_lineup_long_str.split("-")]
        game_stints_df.loc[sub_stint - 1, ['player6_id', 'player7_id', 'player8_id', 'player9_id', 'player10_id']] = OPP_lineup_IDs
        game_stints_df.loc[sub_stint - 1, ['OPP_lineup_long']] = new_OPP_lineup_long_str

    else:
        game_stints_df.loc[sub_stint - 1, ['lineup_long']] = NameError

del(OPP_lineup_IDs, lineup_IDs, new_OPP_lineup_long_str, new_lineup_long_str, period, row, sub_in, sub_index, sub_out, sub_stint)

In [130]:
print(game_stints_df.shape)
game_stints_df.head()

(100, 27)


,Stint_id,start_period,end_period,start_time,end_time,start_seconds,end_seconds,duration_s,start_score,end_score,start_margin,end_margin,plusminus,player1_id,player2_id,player3_id,player4_id,player5_id,lineup_long,lineup_id_ingame,player6_id,player7_id,player8_id,player9_id,player10_id,OPP_lineup_long,OPP_lineup_id_ingame
0,1,1,NaN,12:00,NaN,2880,NaN,NaN,0 - 0,NaN,0,NaN,NaN,1627759,1628369,1630214,201950,1628401,1627759-1628369-1630214-201950-1628401,NaN,203114,203507,201572,201976,203081,203114-203507-201572-201976-203081,NaN
1,2,1,NaN,4:21,NaN,2421,NaN,NaN,11 - 26,NaN,15,NaN,NaN,1627759,1630202,1628436,1630573,1628401,1627759-1630202-1628436-1630573-1628401,NaN,203114,1626171,201572,201976,203081,203114-1626171-201572-201976-203081,NaN
2,3,1,NaN,2:09,NaN,2289,NaN,NaN,17 - 28,NaN,11,NaN,NaN,1627759,1630202,1628436,1630573,1628401,1627759-1630202-1628436-1630573-1628401,NaN,203507,1626171,1627736,1626192,203081,203507-1626171-1627736-1626192-203081,NaN
3,4,1,NaN,0:19,NaN,2179,NaN,NaN,19 - 37,NaN,18,NaN,NaN,1627759,1630202,1628369,1630573,1628401,1627759-1630202-1628369-1630573-1628401,NaN,203507,1626171,1627736,1626192,1641748,203507-1626171-1627736-1626192-1641748,NaN
4,5,2,NaN,12:00,NaN,2160,NaN,NaN,21 - 37,NaN,16,NaN,NaN,1628369,1630214,201950,1630202,1630573,1628369-1630214-201950-1630202-1630573,NaN,203507,1626171,1627736,1626192,1631260,203507-1626171-1627736-1626192-1631260,NaN


In [131]:
"""finalize df: calculate the info for the missing columns (don't need any other dfs but the active game_stints_df, mostly) and last row"""

#shorten df
game_stints_df = game_stints_df[:game_stints_df['Stint_id'].isnull().idxmax()]

#calculate the 'end_' columns that require looking to the next 'start_' rows
game_stints_df['end_period'] = game_stints_df['start_period'].shift(-1)
game_stints_df['end_time'] = game_stints_df['start_time'].shift(-1)
game_stints_df['end_seconds'] = game_stints_df['start_seconds'].shift(-1)
game_stints_df['end_score'] = game_stints_df['start_score'].shift(-1)
game_stints_df['end_margin'] = game_stints_df['start_margin'].shift(-1)

# fill in the last row
game_stints_df.loc[len(game_stints_df)-1, ['end_score']] = pbp2_df[(pbp_df['EVENTMSGTYPE'] == 13) & (pbp2_df['PERIOD'] == 4)]['SCORE'].item()
game_stints_df.loc[len(game_stints_df)-1, ['end_period']] = pbp2_df.loc[len(pbp2_df)-1, ['PERIOD']].item()
game_stints_df.loc[len(game_stints_df)-1, ['end_time']] = '0:00'
game_stints_df.loc[len(game_stints_df)-1, ['end_seconds']] = 0
game_stints_df.loc[len(game_stints_df)-1, ['end_margin']] = int(game_stints_df.loc[len(game_stints_df)-1, ['end_score']].item().partition(' - ')[2]) - \
                                                            int(game_stints_df.loc[len(game_stints_df)-1, ['end_score']].item().partition(' - ')[0])

# calculate the row-wise columns
game_stints_df['duration_s'] = game_stints_df['start_seconds'] - game_stints_df['end_seconds']
#need to change columns to numeric first (should you do this earlier?, are there other columns that need this?)
game_stints_df[["start_margin", "end_margin"]] = game_stints_df[["start_margin", "end_margin"]].apply(pd.to_numeric)
#calculate plusminus
game_stints_df['plusminus'] = game_stints_df['start_margin'] - game_stints_df['end_margin']

In [132]:
print(game_stints_df.shape)
game_stints_df.head()

(20, 27)


,Stint_id,start_period,end_period,start_time,end_time,start_seconds,end_seconds,duration_s,start_score,end_score,start_margin,end_margin,plusminus,player1_id,player2_id,player3_id,player4_id,player5_id,lineup_long,lineup_id_ingame,player6_id,player7_id,player8_id,player9_id,player10_id,OPP_lineup_long,OPP_lineup_id_ingame
0,1,1,1,12:00,4:21,2880,2421,459,0 - 0,11 - 26,0,15,-15,1627759,1628369,1630214,201950,1628401,1627759-1628369-1630214-201950-1628401,NaN,203114,203507,201572,201976,203081,203114-203507-201572-201976-203081,NaN
1,2,1,1,4:21,2:09,2421,2289,132,11 - 26,17 - 28,15,11,4,1627759,1630202,1628436,1630573,1628401,1627759-1630202-1628436-1630573-1628401,NaN,203114,1626171,201572,201976,203081,203114-1626171-201572-201976-203081,NaN
2,3,1,1,2:09,0:19,2289,2179,110,17 - 28,19 - 37,11,18,-7,1627759,1630202,1628436,1630573,1628401,1627759-1630202-1628436-1630573-1628401,NaN,203507,1626171,1627736,1626192,203081,203507-1626171-1627736-1626192-203081,NaN
3,4,1,2,0:19,12:00,2179,2160,19,19 - 37,21 - 37,18,16,2,1627759,1630202,1628369,1630573,1628401,1627759-1630202-1628369-1630573-1628401,NaN,203507,1626171,1627736,1626192,1641748,203507-1626171-1627736-1626192-1641748,NaN
4,5,2,2,12:00,7:30,2160,1890,270,21 - 37,26 - 47,16,21,-5,1628369,1630214,201950,1630202,1630573,1628369-1630214-201950-1630202-1630573,NaN,203507,1626171,1627736,1626192,1631260,203507-1626171-1627736-1626192-1631260,NaN


Notes

-There are NaNs in lineup_id_ingame and OPP_lineup_id_ingame which I intend to fill with lineup set identifiers
(different orders of the same player ids). For now, can delete these columns to remove NaNs

-maybe something involving this:

game_stints_df[['player1_id', 'player2_id', 'player3_id', 'player4_id', 'player5_id']].values.tolist()
game_stints_df.insert(game_stints_df.columns.get_loc('lineup_id_ingame'), 'lineup_set', SOMEKINDOFSET)


# Checking pbp code/data with specific game queries

In the PlayByPlay Data (this game stored in pbp2_df), the game actions occur primarily in 'EVENTMSGTYPE' and 'EVENTMSGACTIONTYPE', the former being primary. First, identify the types of game actions that are tracked.

In [133]:
print(f'EVENTMSGTYPE: {sorted(pbp2_df.EVENTMSGTYPE.unique())}')

EVENTMSGTYPE: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 18]


In [134]:
"""listing EVENTMSGTYPE
https://github.com/swar/nba_api/blob/master/docs/examples/PlayByPlay.ipynb
Actually, I don't know how to do this, here's the enumerated list"""



"listing EVENTMSGTYPE\nhttps://github.com/swar/nba_api/blob/master/docs/examples/PlayByPlay.ipynb\nActually, I don't know how to do this, here's the enumerated list"

    FIELD_GOAL_MADE = 1
    FIELD_GOAL_MISSED = 2
    FREE_THROWfree_throw_attempt = 3
    REBOUND = 4
    TURNOVER = 5
    FOUL = 6
    VIOLATION = 7
    SUBSTITUTION = 8
    TIMEOUT = 9
    JUMP_BALL = 10
    EJECTION = 11
    PERIOD_BEGIN = 12
    PERIOD_END = 13

In [135]:
"""How many minutes did Jaylen Brown play in this game (according to the traditional boxscore)"""

# first get the traditional box score from the game
from nba_api.stats.endpoints import boxscoretraditionalv2
game_boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=single_game_id).get_data_frames()[0]

# get some of Jaylen Brown's row
game_boxscore[game_boxscore['PLAYER_NAME'] == 'Jaylen Brown'][['GAME_ID', 'PLAYER_ID','PLAYER_NAME','MIN','PLUS_MINUS']]


,GAME_ID,PLAYER_ID,PLAYER_NAME,MIN,PLUS_MINUS
0,0022301148,1627759,Jaylen Brown,35.000000:32,-13.0


In [136]:
"""How many minutes did he play using our game_stints_df"""

# using game_stints_df and Jaylen's player id (1627759), how many minutes did Jaylen play?
# which rows have Jaylen in the lineup string
'1627759' in game_stints_df.loc[0]['lineup_long']
game_stints_df['lineup_long'].str.contains('1627759')

# select those rows
Jaylen_in = game_stints_df[game_stints_df['lineup_long'].str.contains('1627759')]
# sum the duration column
Jaylen_in['duration_s'].sum()
# convert to minutes using divmod (division and modulo)
f"{divmod(2132, 60)[0]}:{divmod(2132, 60)[1]}"

'35:32'

In [137]:
"""What was his plus-minus according to game_stints_df"""

Jaylen_in['plusminus'].sum()

-13

In [138]:
"""Middleton's plus-minus (THIS NEEDS WORK, see below)"""

# may need to have every team's roster on hand for player ids etc
# '203114' is Middleton's ID

Middleton_in = game_stints_df[game_stints_df['OPP_lineup_long'].str.contains('203114')]
Middleton_in['plusminus'].sum()

# this is negative when it should be positive, because you need to suss out the Home v away, team v opp, score/margin situation
# maybe there really should be a new pbp_df(3?) where you can have clearer home and visitor? or is it already clear enough
# honestly you should be able to generate the boxscore from the playbyplay

-12

In [139]:
"""Bobby Portis rebounds"""
# 1626171, 4 is the EVENTMSGTYPE for rebounds
len(pbp2_df.query('EVENTMSGTYPE == 4 and PLAYER1_ID == 1626171'))

10

In [140]:
"""JB and JT WOWYs with simple plus-minus"""

# 4 options, JB and JT on, JB on JT off, JT on JB off, both off
# get the lineups where both Jaylens are in (i don't understand this str.contains but i think it works)
game_stints_df[game_stints_df['lineup_long'].str.contains(r'^(?=.*1627759)(?=.*1628369)')]['plusminus'].sum()

-10

In [141]:
#alternatively:
game_stints_df[(game_stints_df['lineup_long'].str.contains('1627759') == True) & (game_stints_df['lineup_long'].str.contains('1628369') == True)]['plusminus'].sum()

-10

In [142]:
#believe this is all right, make a little table
dict = {'Players On': [f"{BOS_roster.query('PERSON_ID == 1628369')['PLAYER_LAST_NAME'].item()}, {BOS_roster.query('PERSON_ID == 1627759')['PLAYER_LAST_NAME'].item()}",
                       f"{BOS_roster.query('PERSON_ID == 1627759')['PLAYER_LAST_NAME'].item()}",
                       f"{BOS_roster.query('PERSON_ID == 1628369')['PLAYER_LAST_NAME'].item()}",
                       ""],
        'Players Off': ["",
                        f"{BOS_roster.query('PERSON_ID == 1628369')['PLAYER_LAST_NAME'].item()}",
                        f"{BOS_roster.query('PERSON_ID == 1627759')['PLAYER_LAST_NAME'].item()}",
                        f"{BOS_roster.query('PERSON_ID == 1628369')['PLAYER_LAST_NAME'].item()}, {BOS_roster.query('PERSON_ID == 1627759')['PLAYER_LAST_NAME'].item()}"],
        'Time': [f"{divmod(game_stints_df[(game_stints_df['lineup_long'].str.contains('1627759') == True) & (game_stints_df['lineup_long'].str.contains('1628369') == True)]['duration_s'].sum(), 60)[0]}:{divmod(game_stints_df[(game_stints_df['lineup_long'].str.contains('1627759') == True) & (game_stints_df['lineup_long'].str.contains('1628369') == True)]['duration_s'].sum(), 60)[1]}",
                    f"{divmod(game_stints_df[(game_stints_df['lineup_long'].str.contains('1627759') == True) & (game_stints_df['lineup_long'].str.contains('1628369') == False)]['duration_s'].sum(), 60)[0]}:{divmod(game_stints_df[(game_stints_df['lineup_long'].str.contains('1627759') == True) & (game_stints_df['lineup_long'].str.contains('1628369') == False)]['duration_s'].sum(), 60)[1]}",
                    f"{divmod(game_stints_df[(game_stints_df['lineup_long'].str.contains('1627759') == False) & (game_stints_df['lineup_long'].str.contains('1628369') == True)]['duration_s'].sum(), 60)[0]}:{divmod(game_stints_df[(game_stints_df['lineup_long'].str.contains('1627759') == False) & (game_stints_df['lineup_long'].str.contains('1628369') == True)]['duration_s'].sum(), 60)[1]}",
                    f"{divmod(game_stints_df[(game_stints_df['lineup_long'].str.contains('1627759') == False) & (game_stints_df['lineup_long'].str.contains('1628369') == False)]['duration_s'].sum(), 60)[0]}:{divmod(game_stints_df[(game_stints_df['lineup_long'].str.contains('1627759') == False) & (game_stints_df['lineup_long'].str.contains('1628369') == False)]['duration_s'].sum(), 60)[1]}"],
        'PlusMinus': [game_stints_df[(game_stints_df['lineup_long'].str.contains('1627759') == True) & (game_stints_df['lineup_long'].str.contains('1628369') == True)]['plusminus'].sum(),
                      game_stints_df[(game_stints_df['lineup_long'].str.contains('1627759') == True) & (game_stints_df['lineup_long'].str.contains('1628369') == False)]['plusminus'].sum(),
                      game_stints_df[(game_stints_df['lineup_long'].str.contains('1627759') == False) & (game_stints_df['lineup_long'].str.contains('1628369') == True)]['plusminus'].sum(),
                      game_stints_df[(game_stints_df['lineup_long'].str.contains('1627759') == False) & (game_stints_df['lineup_long'].str.contains('1628369') == False)]['plusminus'].sum()]}

table = pd.DataFrame(data=dict)
print(table)

     Players On   Players Off   Time  PlusMinus
0  Tatum, Brown                31:30        -10
1         Brown         Tatum    4:2         -3
2         Tatum         Brown   5:56         -1
3                Tatum, Brown   6:32          1


-divmod has a small issue with single digit seconds

-you should just make this into a function with the game_IDs as input (would have to account for lineup_long vs OPP_lineup_long)

-you need more stats per game stint (possessions, pts for, pts against, 3pt shots, rebs etc.)

-should you do this when collecting the stints or use the time to do it post hoc (probably this, i guess you would use SECONDS_LEFT)

-might need to collect the EVENTNUM when making the game stints (seems wisest)